In [0]:
# wavelet transform
# Load per cycle data frame
# This only loaded to get the corresponding labels
import pickle
import pywt
import numpy as np
from skimage.transform import resize


folder = "/content/drive/My Drive/respiratory-sound-database/" 
filename = folder + 'w_c_dataset.pickle'
infile = open(filename,'rb')
[sound,sr,lengths,times,labels] = pickle.load(infile)
infile.close()

In [0]:
def split_sounds(sounds,times,labels):
    s=[]
    l=[]
    for i,sound in enumerate(sounds):
        for t,label in zip(times[i],labels[i]):
            s.append(sound[int(t[0]):int(t[1])])
            if label==0:
                a=np.array([1,0,0,0])
            if label==1:
                a=np.array([0,1,0,0])
            if label==2:
                a=np.array([0,0,1,0])    
            if label==3:
                a=np.array([0,0,0,1])
            l.append(a)
    return s,l

In [0]:
def wavelet_transform(d):
  scales = np.arange(1,41)
  wavelet = 'morl'
  coeffs, freqs = pywt.cwt(d, scales, wavelet = wavelet)
  rescale_coeffs = resize(coeffs, (40,40), mode = 'constant')
  mean_wavelet = np.mean(rescale_coeffs.T,axis=0)
  std_wavelet = np.std(rescale_coeffs.T,axis=0)
  vector = np.hstack((mean_wavelet,std_wavelet))
  return vector, rescale_coeffs

In [0]:
[data,label]=split_sounds(sound,times,labels)

In [0]:
wavelets = []; wavelet_matrices = []
for d in range(len(data)):
  print(d)
  v,m = wavelet_transform(data[d])
  wavelets.append(v)
  wavelet_matrices.append(m)
wavelets = np.concatenate(wavelets, axis=0)

In [0]:
import pandas as pd
import pickle

data_dict = {}
pickle_out = open("/content/drive/My Drive/respiratory-sound-database/wavelet.pickle","wb")
d_dict = {}
d_dict['w'] = wavelets
d_dict['w_m'] = wavelet_matrices
pickle.dump(data_dict,pickle_out)
pickle_out.close()

In [0]:
import pandas as pd
import pickle

pickle_in = open("/content/drive/My Drive/respiratory-sound-database/wavelet.pickle","rb")
data = pickle.load(pickle_in)
dfr = data['w']
df = pd.read_csv('/content/drive/My Drive/respiratory-sound-database/Respiratory_Data.csv')
pickle_in.close()
labels = dfr['Class'].to_numpy()

In [0]:
one=0; two=0;three=0;four=0;
for i in range(labels.shape[0]):
  if labels[i] == 0:
    one = one + 1
  elif labels[i] == 1:
    two = two + 1
  elif labels[i] == 2:
    three = three + 1
  else:
    four = four + 1

le = LabelEncoder()
i_labels = le.fit_transform(labels)

In [0]:
scaler=StandardScaler()
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(X,labels,test_size=0.3, random_state=42)
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [0]:
# Running SVM classifier
Cs = [2**(-2),2**(-1), 1,2**(1),2**(2),2**(3),2**(4),2**(5),2**(6),2**(7),2**(8),2**(9),2**(10)]
gamma = [2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2**(0),2**(1),2**(2),2**(3)]


param_grid = {'C': Cs,  
                'gamma': gamma, 
                'kernel': ['rbf'],
                'decision_function_shape':['ovr'],
                'class_weight': ['balanced']}  
grid1 = GridSearchCV(SVC(), param_grid,cv=3,n_jobs=-1, verbose = 3) 

# fitting the model for grid search 
grid1.fit(x_train, y_train)

# print best parameter after tuning 
print(grid1.best_params_) 
# print how our model looks after hyper-parameter tuning 
print(grid1.best_estimator_)

grid_predictions = grid1.predict(x_test) 

# print classification report 
print(classification_report(y_test, grid_predictions))
print(accuracy_score(y_test,grid_predictions))

print(sklearn.metrics.confusion_matrix(y_test,grid_predictions))

In [0]:
# Running XgBoost
import xgboost as xgb

print('Training XGB Classifier from new features:')
xgb_model = xgb.XGBClassifier(max_depth=3,num_class=2, n_estimators=15, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='multi:softmax', eta=0.3, silent=0, subsample=0.8).fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_test)
print(accuracy_score(y_test,xgb_pred))

In [0]:
#Running ANN
# One hot the labels
le = LabelEncoder()
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)



# Custom model for classification
model = Sequential()

model.add(Dense(24, activation='relu', input_shape=(24,),kernel_initializer='random_normal'))

model.add(Dense(12, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(300, activation='relu'))

#model.add(Dense(150, activation='relu')) 
#model.add(Dropout(0.2))
#model.add(Dense(75, activation='relu')) 
#model.add(Dropout(0.2))
#model.add(Dense(64, activation='relu',kernel_initializer='random_normal')) 
#model.add(Dropout(0.2))


#model.add(Dense(16, activation='relu') )

model.add(Dense(2, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam') 

model.summary()



# Load the trained model if required
#model_file = '/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/LogMel_MFCC_Chroma_CNN/Try6_Unlinked/model_40_230.h5'
#model.load_weights(model_file)

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)